# Solution de classification flèche/cible

Les fichiers fournis par l'équipe de Medvir sont complet et regroupent toutes les combinaisons possible de flèche + cible désignant un symptome.

In [55]:
# Importe spaCy
import spacy
import json
from spacy.matcher import PhraseMatcher

from Levenshtein import distance

# Crée un objet nlp français vide
nlp = spacy.blank("fr")

# Charge le moyen pipeline français
nlp = spacy.load("fr_core_news_md")

In [56]:
# Crée en traitant une chaine de caractères avec l'objet nlp
doc = nlp("J'ai mal au ventre et une boule au niveau du nombril.")

On récupère le vocabulaire.

In [57]:
with open("fleche-cible.json", encoding="utf8") as f:
    VOCABULAIRE = json.loads(f.read())

print(VOCABULAIRE)

{'flèche': {'absence': ['absence', 'pas de'], 'anomalie': ['plissé', 'flasque', 'étiré', 'élastique', 'anormale', 'anomalie', 'sclérosé', 'sclérose', 'plissement', 'étirement', 'élasticité'], 'arrêt': ['arrêt', 'blocage', 'bloqué', 'occlusion', 'subocclusion', 'obstacle', 'encombré', 'encombrement', 'empêchement', 'entrave'], 'bleu': ['bleu', 'ecchymose', 'hématome'], 'blocage': ['blocage', 'bloqué', 'coincé', 'coincement'], 'boule': ['boule'], 'boutons': ['anneau', 'plaque', 'macule', 'papule', 'vésicule', 'bulle', 'tache', 'urticaire', 'bubon', 'phlyctène', 'rash', 'éruption', 'abcès', 'urticaire', 'phlegmon', 'anthrax', 'cloque'], 'bruit': ['borborygmes', 'bruit', 'gargouillement', 'gargouillis', 'grincement', 'crissement', 'craquement', 'bruissement', 'détonation', 'explosion', 'éclatement', 'froissement', 'glouglou', 'grésillement', 'râle sifflement', 'tintement', 'vacarme', 'murmure'], 'brulure': ['brulure', 'brulé', 'cramé'], 'choc': ['baffe', 'choc', 'claque', 'cogné', 'contusi

On va créer un dictionnaire qui va nous permettre de receuillir toutes les informations dont on a besoin.

In [58]:
dictionnaire = {}
text_reduit = []
for token in doc :
    for categorie, mots_associes in VOCABULAIRE.items():
        for index, (mot_cle, synonymes) in enumerate(mots_associes.items()):
            for synonyme in synonymes:
                if str(token.lemma_) == synonyme : # regarde si le mot est dans le vocabulaire médicale
                    dictionnaire[str(token.lemma_)] = {} # création dictionnaire 
                    dictionnaire[str(token.lemma_)]["catégorie"] = categorie # ajout de la catégorie du mot
                    dictionnaire[str(token.lemma_)]["index"] = index # ajout de l'indice du mot
                    text_reduit.append(str(token.lemma_))

print(dictionnaire)


{'mal': {'catégorie': 'flèche', 'index': 18}, 'ventre': {'catégorie': 'zone', 'index': 2}, 'boule': {'catégorie': 'flèche', 'index': 5}, 'nombril': {'catégorie': 'zone', 'index': 19}}


On a créer notre dictionnaire avec les informations maintenant on va pourvoir les traiter plus facillement.

In [59]:
codes = []

for i, entite in enumerate(text_reduit): # on associe chaque flèche avec sa cible (la plus proche physiquement)
    if (i != len(text_reduit)-1):
        if dictionnaire[entite]["catégorie"] == "flèche" and (dictionnaire[text_reduit[i+1]]["catégorie"] == "zone" or dictionnaire[text_reduit[i+1]]["catégorie"] == "organe" or dictionnaire[text_reduit[i+1]]["catégorie"] == "fonction"):
            if dictionnaire[text_reduit[i+1]]["catégorie"] == "zone":
                codes.append([dictionnaire[entite]["index"], dictionnaire[text_reduit[i+1]]["index"], "*", "*"])
            if dictionnaire[text_reduit[i+1]]["catégorie"] == "organe":
                codes.append([dictionnaire[entite]["index"], "*", dictionnaire[text_reduit[i+1]]["index"], "*"])
            if dictionnaire[text_reduit[i+1]]["catégorie"] == "fonction":
                codes.append([dictionnaire[entite]["index"], "*", "*", dictionnaire[text_reduit[i+1]]["index"]])
            text_reduit.pop(i+1)
        elif (dictionnaire[entite]["catégorie"] == "zone" or dictionnaire[entite]["catégorie"] == "organe" or dictionnaire[entite]["catégorie"] == "fonction") and dictionnaire[text_reduit[i+1]]["catégorie"] == "flèche":
            if dictionnaire[entite]["catégorie"] == "zone":
                codes.append([dictionnaire[text_reduit[i+1]]["index"], dictionnaire[entite]["index"], "*", "*"])
            if dictionnaire[entite]["catégorie"] == "organe":
                codes.append([dictionnaire[text_reduit[i+1]]["index"], "*", dictionnaire[entite]["index"], "*"])
            if dictionnaire[entite]["catégorie"] == "fonction":
                codes.append([dictionnaire[text_reduit[i+1]]["index"], "*", "*", dictionnaire[entite]["index"]])
            text_reduit.pop(i+1)
        

print(codes)

[[18, 2, '*', '*'], [5, 19, '*', '*']]


Un fois qu'on à récupérer les codes on peut associer les codes à leur symptome.

In [60]:
with open("resultats.json", encoding="utf8") as f:
    RESULT = json.loads(f.read())

print(RESULT)

{'4': {'label': 'Articulation déboitée', 'resultat': [[14, 0, '*', '*'], [14, 1, '*', '*'], [14, 29, '*', '*'], [14, 3, '*', '*'], [14, 4, '*', '*'], [14, 5, '*', '*'], [14, 6, '*', '*'], [14, 7, '*', '*'], [14, 8, '*', '*'], [14, 9, '*', '*'], [14, 10, '*', '*'], [14, 12, '*', '*'], [14, 13, '*', '*'], [14, 14, '*', '*']]}, '5': {'label': 'Articulations gonflées', 'resultat': [[22, 0, '*', '*'], [22, 1, '*', '*'], [22, 29, '*', '*'], [22, 3, '*', '*'], [22, 4, '*', '*'], [22, 5, '*', '*'], [22, 6, '*', '*'], [22, 7, '*', '*'], [22, 8, '*', '*'], [22, 9, '*', '*'], [22, 10, '*', '*'], [22, 12, '*', '*'], [22, 13, '*', '*'], [22, 14, '*', '*'], [22, '*', '*', 1]]}, '6': {'label': 'Ballonnement', 'resultat': [[22, 2, '*', '*'], [22, '*', 4, '*'], [22, '*', 5, '*']]}, '0': {'label': '', 'resultat': [[3, 15, '*', '*'], [3, '*', 19, '*'], [3, 16, '*', '*'], [4, '*', '*', 2], [17, '*', '*', 2], [0, '*', '*', 2], [4, '*', 5, '*'], [4, '*', '*', 3], [15, '*', 22, '*'], [17, '*', '*', 7]]}, '8'

In [61]:
symptomes_med = []

for c in codes :
    for code, symptomes in RESULT.items() :
        if c in symptomes["resultat"] :
            print("Symptome : "+ symptomes["label"] +", de code "+code)
            symptomes_med.append(code)

print(symptomes_med)

Symptome : Mal au ventre, de code 8
Symptome : Boule au nombril, de code 13
['8', '13']


On obtient bien le resultat attendu. On peut même garder une trace en créant un fichier json avec le dictionnaire ce qui pourrait nous permettre de créer une sauvegarde des plaintes patient. 